# 数据读取与转换

## 2.1 数据集的描述

+ 本项目包含客户在某移动支付平台的367万条脱敏交易流水数据，交易时间跨度为5年。
+ 每个客户有多条交易流水，每条交易流水记录了客户ID、交易金额、交易附言和交易时间四个字段。
+ 各字段的英文名称、中文名称和备注如表2.1所示：

<center><font size="2.5px" face="微软雅黑" color="666666">表2.1：客户交易流水记录字段说明</font></center>
<center><img src=".\Pics\Table2_1.jpg" width="750" height="200" alt="cursor" align=center /></center>

## 2.2 从MySQL数据库中读取交易流水数据

本项目的交易流水记录存储在数据库`transactions`的`flow_data`表中。`flow_data`表的字段描述如表2.2所示：

<center><font size="2.5px" face="微软雅黑" color="666666">表2.2：flow_data数据表结构</font></center>
<center><img src=".\Pics\Table2_2.jpg" width="250" height="125" alt="cursor" align=center /></center>

为了便于分析数据，我们使用`pymysql`模块连接数据库并导出数据。

数据库连接对象`connection`
+ 作用：建立Python客户端与数据库的连接。
+ 创建方法：`connection=pymysql.connect(参数)`

数据库游标对象`cursor`
+ 作用：用于执行查询和获取结果。
+ 创建方法：`cursor = connection.cursor()`。
+ 在MySQL中，所有`SQL`语句的执行都要在游标对象的参与下完成。

<center><img src=".\Pics\Pic2_1.jpg" width="500" height="300" alt="cursor" align=center /></center>
<center><font size="2.5px" face="微软雅黑" color="666666">图2.1：数据库连接步骤</font></center>
<!-- ![图2.1：数据库连接步骤](Pic2_1.jpg) -->

>### 实训任务
>+  请补充完成创建数据库连接对象`connection`的语句。
>+  请书写`SQL`语句并赋予变量`sql1`，从数据库中查询表`flow_data`的行数。
>+  请书写`SQL`语句并赋予变量`sql2`，从数据库中查询表`flow_data`的前五行。

提示：`connection`对象中的参数为：
 + `host`：MySQL服务器地址。
 + `port`：连接的端口号，MySQL数据库默认端口是`3306`。
 + `user`和`password`：登陆数据库的用户名和密码。
 + `db`：要连接的数据库名字。
 + `charset`：字符编码，我们设置为`utf8mb4`，因为它可以很好地支持中文。
 
 
表2.3显示了创建连接对象`connection`所需要的参数信息：

<center><font size="2.5px" face="微软雅黑" color="666666">表2.3：MySQL连接参数表</font></center>

|参数名称|参数信息|参数类型|
|:----|:-----|:----|
|host|traindb.cookdata.cn|string|
|port|3306|int|
|user|raa_user|string|
|password|bigdata123|string|
|db|transactions|string|
|charset|utf8mb4|string|

可以看到数据行数为3672588，使用游标对象读取出的数据以嵌套元组的方式进行存储。

In [ ]:
import pymysql

# 建立连接对象
connection = pymysql.connect(
    host = 'traindb.cookdata.cn',
    port = 3306,
    user = 'raa_user',
    password = 'bigdata123',
    db = 'transactions',
    charset = 'utf8mb4',
)

try:
    with connection.cursor() as cursor:
        # 查询数据表行数
        sql1 = "SELECT count(*) FROM flow_data"
        # 选取表的前五行
        sql2 = "SELECT * FROM flow_data LIMIT 5"

        cursor.execute(sql1)
        rows = cursor.fetchall() 
        cursor.execute(sql2)
        head = cursor.fetchall() 

finally:
    connection.close()

# 查看数据的行列数
print('数据行数为:',rows,'\n前五行数据为:',head)

## 2.2 对客户数据进行格式转换

在上一步中，我们已经成功地在数据库中进行了一些查询和读取操作，现在我们需要利用Python中的Pandas库进行数据格式的转换。

题目已预先从数据表中读入全部数据(`SELECT * FROM flow_data`)，并命名它为`data`。但目前的数据是以嵌套元组的形式存储的，所以需要先将其转换为Pandas内置的DataFrame对象，再进行后续操作。

DataFrame对象是Pandas最常用的数据对象。DataFrame是一个二维的表结构，它可以存储多种不同的数据类型。

使用Pandas中的`DataFrame()`函数可以创建一个DataFrame对象。函数语法为：`DataFrame(data, index, columns, dtype)`，其中：
+ `data`：输入的数据，可以是各种形式，比如ndarray、Series、map、list、dict、constant和另一个DataFrame。 
+ `index`：设置行索引。
+ `columns`：设置列名。
+ `dtype`：设置每列的数据类型。

嵌套元组不能直接保存为DataFrame，需要将外层的元组先转换为列表，再使用`DataFrame()`函数将列表进行转换。但是直接转换时，列名为默认值（即0、1、2、3），所以需要在转换时重命名列名。

>### 实训任务
>+ 请将预先读入的数据（已命名为`data`）转换为DataFrame的形式，保存在变量`data`中。
>  + 提示：
>        首先使用`list()`函数将嵌套元组转换为列表，再使用`DataFrame()`函数将列表转换为DataFrame形式，转换时需要设置参数`columns=['user_id', 'payment', 'describe', 'unix_time']`。

In [ ]:
import pandas as pd

# 将 data 转换为 DataFrame 格式
data = pd.DataFrame(list(data), columns=['user_id', 'payment', 'describe', 'unix_time'])

print(data.head())